In [ ]:
import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp

from math import pi
from numpy import linalg as LA
import matplotlib.pyplot as plt

### Erstellt ein Quadrat mit der Länge $1$ 
### und setzt einen Impuls in der Mitte des Quadrats

In [ ]:
import netgen.geom2d as geom2dim
geo = geom2dim.SplineGeometry()
a = 1
points = [(0,0),(a,0),(a,a),(0,a)]
p1,p2,p3,p4 = [geo.AppendPoint(*pnt) for pnt in points]
curves = [['line',p1,p2],['line',p2,p3],['line',p3,p4],['line',p4,p1]]
[geo.Append(c) for c in curves]

mesh = Mesh(geo.GenerateMesh(maxh = 0.1))


piha = a/2
shift = 0
source = exp(-50**2*((x-piha-shift)*(x-piha-shift)+(y-piha-shift)*(y-piha-shift)))
Draw(source,mesh,name = "pulse")

In [ ]:
maxind = 20
#fes = H1(mesh, order=5)
fes = H1(mesh, order=5,dirichlet=".*")
gfu = GridFunction(fes,name = "u")
erg = np.zeros([len(gfu.vec),maxind])
fh = np.zeros(len(gfu.vec))

### The gereral solution form the helmholz equation is 
$u = \sin(l \pi x)\sin(k \pi y)$

when we plug now this ansatz in the helmholz equation:

$k^2\pi^2u+l^2\pi^2u -\omega^2u = 0$

For $u \neq 0$ it follows that:

$k^2\pi^2+l^2\pi^2 = \omega^2$ $\Rightarrow$ $\omega = \sqrt{k^2\pi^2+l^2\pi^2}$ where $k,l \in \mathbb{N}$

In [ ]:
verg = []
point = []
j = 0
temp = 0
for k in range(1,20):
    for l in range(1,20):
        temp = np.sqrt(l**2*pi**2+k**2*pi**2)
        if l > 1:
            if temp == verg[j-1]:
                temp = 0
        else:
            verg.append(temp)
    j = j+1
            
        
verg.sort()       
print(verg)

### Die helmholzgleichung:
$\Delta u - \omega^2 u = f$
### und die Schwache Formulierung:
$\int_{\Omega}\nabla u \nabla v - \omega^2\int_{\Omega}u v = \int_{\Omega}fv$ $v \in V$
### löse die obrige Gleichung mit paar $\omega_i$ und speichere diese $u_i$ in eine Matrix 

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
K = BilinearForm(fes)
M = BilinearForm(fes)
F = LinearForm(fes)

omega = Parameter(1)
A += (grad(u)*grad(v)- omega**2*u*v)*dx
K += grad(u)*grad(v)*dx
M += u*v*dx


#A += 1j*omega*u*v*ds # R 
F += source*v*dx  
o = []  

norm = []

#o = verg[0:21]
for i in range(0,maxind):
    o.append(i)

j = 0
Draw(gfu)
for i in o:   
    omega.Set(i) 
    A.Assemble()
    K.Assemble()
    M.Assemble()
    F.Assemble()   
    gfu.vec.data = A.mat.Inverse(fes.FreeDofs()) * F.vec
    erg[:,j] = gfu.vec
    fh = F.vec
    norm.append(sqrt (Integrate ((gfu)*(gfu),mesh)))
    print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((gfu)*(gfu),mesh)))
    j = j+1
As = A.mat
Ms = M.mat
Ks = K.mat
rows,cols,vals = Ms.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = Ks.COO()
Kh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = As.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Ah = sp.csr_matrix.toarray(Ah)

### calulates the orthogonal basis matrix V (using qr)

In [ ]:
V, r = np.linalg.qr(erg)
V = erg.dot(np.linalg.inv(r))
print(V[:,1].dot(V[:,2]))
len(V)
#lam,v = LA.eig(Ah)
#q,R = np.linalg.qr(Ah)
#print(np.diag(R))

### Eigenvalue calculation with eig and qr (very expensive)

In [ ]:
q,R = np.linalg.qr(Ah)

In [ ]:
#print("from eig: ",lam[0:20])
eig = []
eig = np.diag(R)*np.diag(R)
for i in verg:
    for j in range(len(eig)):
        #print(np.abs(i-eig[j]))
        if np.abs(i-eig[j]) < 0.1:
            print(j,eig[j])
#print("from qr: ",eig[0:1000])

### Plot the snapshot solution 

In [ ]:
plt.plot(o[1:20],norm[1:20])
plt.xlabel("omega")
plt.ylabel("Euklidic-Norm")
plt.savefig('results/resonanz_20.png')

### Für die erste Metode GramSchmid verwenden um eine orthonormale Koordinatentransformationsmatrix zu erstellen(Euklidische Norm). 

$$
M_N = V^T \cdot M_h \cdot V
$$

$$
K_N = V^T \cdot K_h \cdot V
$$

$$
f_N = V^T \cdot f_h
$$

$$
u_i = (K_N - \omega^2_iM_N)^{-1} \cdot f_N
$$

In [ ]:
MN = np.transpose(V).dot(Mh.dot(V))
KN = np.transpose(V).dot(Kh.dot(V))
fN = np.transpose(V).dot(fh)

### calculate the res, red_solution and res_red snapshot solution 

In [ ]:
j = 0
uN = np.zeros([maxind,maxind])
res_red_fh = []
res_red_fN = []
for fre in o:
    uN[:,j] = np.linalg.solve(KN-fre**2*MN, fN)
    res_red_fh.append(np.linalg.norm((Kh-fre**2*Mh).dot(uN[:,j].dot(np.transpose(V)))-fh))
    res_red_fN.append(np.linalg.norm((Kh-fre**2*Mh).dot(uN[:,j].dot(np.transpose(V)))-fN.dot(np.transpose(V))))
    j = j+1

In [ ]:
plt.semilogy(o[1:19],res_red_fh[1:19])
plt.xlabel("omega")
plt.ylabel("res_red_fh")
plt.savefig('results/resonanz_RB_20.png')

In [ ]:
plt.semilogy(o[1:19],res_red_fN[1:19])
plt.xlabel("omega")
plt.ylabel("res_red_fN")
plt.savefig('results/resonanz_RB_20.png')

In [ ]:
iteration = int(100)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    omegaN.append(factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    new = uN[:,i].dot(np.transpose(V))
    gif = GridFunction(fes,name = "gif")
    gif.vec.FV().NumPy()[:] = new
    Draw(gif)
    print("i: ",i,"Omega: ",omegaN[i])
    input()
    Redraw()
    #norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))

### calculates 1000 residuals and the norm of the rediced solution (very expensive)

In [ ]:
verg = []
point = []
for k in range(1,100):
    for l in range(1,100):
        verg.append(np.sqrt(l**2*pi**2+k**2*pi**2))
        point.append(0.1)


iteration = int(1e4)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
res = []
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    omegaN.append(factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    res.append(np.linalg.norm((Kh-omegaN[i]**2*Mh).dot(uN[:,i].dot(np.transpose(V)))-fN.dot(np.transpose(V))))
    norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))
    
    

In [ ]:
maxpoints = 6
plt.semilogy(omegaN,norm_L2)
plt.semilogy(verg[0:maxpoints],point[0:maxpoints],'*r')
plt.semilogy(omegaN,norm_L2,'b')
plt.xlabel("omega")
plt.ylabel("Euclidic Norm")
plt.savefig('results/resonanz_RB_20.png')

In [ ]:
print(verg[0:7])

In [ ]:
plt.semilogy(omegaN,res)
plt.xlabel("omega")
plt.ylabel("Euclidic Norm")
plt.savefig('results/resonanz_RB_20.png')

In [ ]:
newom = []
for i in range(len(res)):
    if res[i] < 6.94e-4:
        print(i,omegaN[i],res[i])
        newom.append(omegaN[i])
    